In [1]:
import numpy as np
import os
import sys
from ultralytics import YOLO
from PIL import Image
import torch
from torch import tensor
from torchvision.transforms import transforms, InterpolationMode
import timm
import torch.nn as nn
import pandas as pd

ERROR Error reading from C:\Users\plk\AppData\Roaming\Ultralytics\settings.json: "No Ultralytics setting 'openvino_msg'. \nView Ultralytics Settings with 'yolo settings' or at 'C:\\Users\\plk\\AppData\\Roaming\\Ultralytics\\settings.json'\nUpdate Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings."
Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\plk\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


d:\Pobrane\mgr\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DFPATH = 'models'
DFYOLO_NAME = "deepfaune-yolov8s_960"
BACKBONE = "vit_large_patch14_dinov2.lvd142m"
DFYOLO_WEIGHTS = os.path.join(DFPATH,'deepfaune-yolov8s_960.pt')
DFVIT_WEIGHTS = os.path.join(DFPATH,'deepfaune-vit_large_patch14_dinov2.lvd142m.v3.pt')

#### Setup

In [3]:
CROP_SIZE = 182 # default 182, has to be divisible by 14, 280?
DFYOLO_WIDTH = 960 # image width
DFYOLO_THRES = 0.6
DFYOLOHUMAN_THRES = 0.4 # boxes with human above this threshold are saved
DFYOLOCOUNT_THRES = 0.6

txt_animalclasses = {
   'en': ['bison', 'badger', 'ibex', 'beaver', 'red deer', 'chamois', 'cat', 'goat', 'roe deer', 'dog', 'fallow deer', 'squirrel', 'moose', 'equid', 'genet', 'wolverine', 'hedgehog', 'lagomorph', 'wolf', 'otter', 'lynx', 'marmot', 'micromammal', 'mouflon', 'sheep', 'mustelid', 'bird', 'bear', 'nutria', 'raccoon', 'fox', 'reindeer', 'wild boar', 'cow'],
   'pl': ['żubr', 'borsuk', 'koziorożec', 'bóbr', 'jeleń szlachetny', 'kozica', 'kot', 'koza', 'sarna', 'pies', 'daniel', 'wiewiórka', 'łoś', 'końowaty', 'żeneta', 'rosomak', 'jeż', 'zajęczak', 'wilk', 'wydra', 'ryś', 'świstak', 'mikrozwierzę', 'muflon', 'owca', 'łasica', 'ptak', 'niedźwiedź', 'nutria', 'szop', 'lis', 'renifer', 'dzik', 'krowa']
    #'fr': ['bison', 'blaireau', 'bouquetin', 'castor', 'cerf', 'chamois', 'chat', 'chevre', 'chevreuil', 'chien', 'daim', 'ecureuil', 'elan', 'equide', 'genette', 'glouton', 'herisson', 'lagomorphe', 'loup', 'loutre', 'lynx', 'marmotte', 'micromammifere', 'mouflon', 'mouton', 'mustelide', 'oiseau', 'ours', 'ragondin', 'raton laveur', 'renard', 'renne', 'sanglier', 'vache'],
    #'it': ['bisonte', 'tasso', 'stambecco', 'castoro', 'cervo', 'camoscio', 'gatto', 'capra', 'capriolo', 'cane', 'daino', 'scoiattolo', 'alce', 'equide', 'genetta', 'ghiottone', 'riccio', 'lagomorfo', 'lupo', 'lontra', 'lince', 'marmotta', 'micromammifero', 'muflone', 'pecora', 'mustelide', 'uccello', 'orso', 'nutria', 'procione', 'volpe', 'renna', 'cinghiale', 'mucca'],
    #'de': ['Bison', 'Dachs', 'Steinbock', 'Biber', 'Rothirsch', 'Gämse', 'Katze', 'Ziege', 'Rehwild', 'Hund', 'Damwild', 'Eichhörnchen', 'Elch', 'Equide', 'Ginsterkatze', 'Vielfraß', 'Igel', 'Lagomorpha', 'Wolf', 'Otter', 'Luchs', 'Murmeltier', 'Kleinsäuger', 'Mufflon', 'Schaf', 'Marder', 'Vogel', 'Bär', 'Nutria', 'Waschbär', 'Fuchs', 'Rentier', 'Wildschwein', 'Kuh'],
}

class Detector:
    def __init__(self, name=DFYOLO_NAME, threshold=None, countthreshold=None, humanthreshold=None):
        print("Using "+DFYOLO_NAME+" with weights at "+DFYOLO_WEIGHTS+", in resolution 960x960")
        self.yolo = YOLO(DFYOLO_WEIGHTS)
        self.imgsz = DFYOLO_WIDTH
        self.threshold = DFYOLO_THRES if threshold is None else threshold
        self.countthreshold = DFYOLOCOUNT_THRES if countthreshold is None else countthreshold
        self.humanthreshold = DFYOLOHUMAN_THRES if humanthreshold is None else humanthreshold

    def bestBoxDetection(self, filename_or_imagecv):
        try:
            results = self.yolo(filename_or_imagecv, verbose=False, imgsz=self.imgsz)
        except FileNotFoundError:
            print("File not found")
            return None, 0, np.zeros(4), 0, []
        except Exception as err:
            print(err)
            return None, 0, np.zeros(4), 0, []

        # orig_img a numpy array (cv2) in BGR
        imagecv = results[0].cpu().orig_img
        detection = results[0].cpu().numpy().boxes

        # Are there any relevant boxes?
        if not len(detection.cls) or detection.conf[0] < self.threshold:
            # No. Image considered as empty
            return None, 0, np.zeros(4), 0, []
        else:
            # Yes. Non empty image
            pass
        # Is there a relevant animal box? 
        try:
            # Yes. Selecting the best animal box
            kbox = np.where((detection.cls==0) & (detection.conf>self.threshold))[0][0]
        except IndexError:
            # No: Selecting the best box for another category (human, vehicle)
            kbox = 0

        # categories are 1=animal, 2=person, 3=vehicle and the empty category 0=empty
        category = int(detection.cls[kbox]) + 1
        box = detection.xyxy[kbox] # xmin, ymin, xmax, ymax

        # Is this an animal box ?
        if category == 1:
            # Yes: cropped image is required for classification
            croppedimage = cropSquareCVtoPIL(imagecv, box.copy())
        else: 
            # No: return none
            return None, 0, np.zeros(4), 0, []
        
        ## animal count
        if category == 1:
            count = sum((detection.conf>self.countthreshold) & (detection.cls==0)) # only above a threshold
        else:
            count = 0
        ## human boxes
        ishuman = (detection.cls==1) & (detection.conf>=self.humanthreshold)
        if any(ishuman==True):
            humanboxes = detection.xyxy[ishuman,]
        else:
            humanboxes = []

        return croppedimage, category, box, count, humanboxes


def cropSquareCVtoPIL(imagecv, box):
    x1, y1, x2, y2 = box
    xsize = (x2-x1)
    ysize = (y2-y1)
    if xsize>ysize:
        y1 = y1-int((xsize-ysize)/2)
        y2 = y2+int((xsize-ysize)/2)
    if ysize>xsize:
        x1 = x1-int((ysize-xsize)/2)
        x2 = x2+int((ysize-xsize)/2)
    height, width, _ = imagecv.shape
    croppedimagecv = imagecv[max(0,int(y1)):min(int(y2),height),max(0,int(x1)):min(int(x2),width)]
    croppedimage = Image.fromarray(croppedimagecv[:,:,(2,1,0)]) # converted to PIL BGR image
    return croppedimage


class Classifier:

    def __init__(self):
        self.model = Model()
        self.model.loadWeights(DFVIT_WEIGHTS)
        # transform image to form usable by network
        self.transforms = transforms.Compose([
            transforms.Resize(size=(CROP_SIZE, CROP_SIZE), interpolation=InterpolationMode.BICUBIC, max_size=None, antialias=None),
            transforms.ToTensor(),
            transforms.Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))])

    def predictOnBatch(self, batchtensor, withsoftmax=True):
        return self.model.predict(batchtensor, withsoftmax)

    # croppedimage loaded by PIL
    def preprocessImage(self, croppedimage):
        preprocessimage = self.transforms(croppedimage)
        return preprocessimage.unsqueeze(dim=0)

class Model(nn.Module):
    def __init__(self):
        """
        Constructor of model classifier
        """
        super().__init__()
        self.base_model = timm.create_model(BACKBONE, pretrained=False, num_classes=len(txt_animalclasses['en']),
                                            dynamic_img_size=True)
        print(f"Using {BACKBONE} with weights at {DFVIT_WEIGHTS}, in resolution {CROP_SIZE}x{CROP_SIZE}")
        self.backbone = BACKBONE
        self.nbclasses = len(txt_animalclasses['en'])

    def forward(self, input):
        x = self.base_model(input)
        return x

    def predict(self, data, withsoftmax=True):
        """
        Predict on test DataLoader
        :param test_loader: test dataloader: torch.utils.data.DataLoader
        :return: numpy array of predictions without soft max
        """
        self.eval()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(device)
        total_output = []
        with torch.no_grad():
            x = data.to(device)
            if withsoftmax:
                output = self.forward(x).softmax(dim=1)
            else:
                output = self.forward(x)
            total_output += output.tolist()

        return np.array(total_output)

    def loadWeights(self, path):
        """
        :param path: path of .pt save of model
        """
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("CUDA available" if torch.cuda.is_available() else "CUDA unavailable. Using CPU")

        try:
            params = torch.load(path, map_location=device)
            args = params['args']
            if self.nbclasses != args['num_classes']:
                raise Exception("You load a model ({}) that does not have the same number of class"
                                "({})".format(args['num_classes'], self.nbclasses))
            self.backbone = args['backbone']
            self.nbclasses = args['num_classes']
            self.load_state_dict(params['state_dict'])
        except Exception as e:
            print("Can't load checkpoint model because :\n\n " + str(e), file=sys.stderr)
            raise e


#### Detect animal and crop image

In [5]:
IMAGES_PATH = 'test_images/'
IMAGE_FILE = '2022-09-06 19-34-11.JPG'

det = Detector()

Using deepfaune-yolov8s_960 with weights at models\deepfaune-yolov8s_960.pt, in resolution 960x960


In [9]:
croppedimage, category, box, count, humanboxes = det.bestBoxDetection(IMAGES_PATH + IMAGE_FILE)

#### Classify animal

In [7]:
classifier = Classifier()

Using vit_large_patch14_dinov2.lvd142m with weights at models\deepfaune-vit_large_patch14_dinov2.lvd142m.v3.pt, in resolution 182x182
CUDA available


In [10]:
cropped_tensor = torch.ones((1,3,CROP_SIZE,CROP_SIZE))
cropped_tensor[0,:,:,:] =  classifier.preprocessImage(croppedimage)
scores = classifier.model.predict(cropped_tensor)
scores

TypeError: Unexpected type <class 'NoneType'>

In [ ]:
output = pd.DataFrame({'species':txt_animalclasses['pl'], 'score':scores[0][:]})
output = output[output['score'] > 0.2]

In [ ]:
print(f"Wykryto zwierzę: {output.loc[output['score'].idxmax(), 'species']}")

In [ ]:
croppedimage

#### Pipeline klasyfikacji

In [ ]:
detector = Detector()
classifier = Classifier()

In [ ]:
IMAGES_PATH = 'test_images/'
IMAGE_FILE = '2023-05-01 00-04-04.JPG'

croppedimage, category, box, count, humanboxes = detector.bestBoxDetection(IMAGES_PATH + IMAGE_FILE)
if croppedimage is not None:
    cropped_tensor = torch.ones((1,3,CROP_SIZE,CROP_SIZE))
    cropped_tensor[0,:,:,:] =  classifier.preprocessImage(croppedimage)
    scores = classifier.model.predict(cropped_tensor)
    output = pd.DataFrame({'species':txt_animalclasses['pl'], 'score':scores[0][:]})
    output = output[output['score'] > 0.2]
    print(output)

    print(f"\nWykryto zwierzę: {output.loc[output['score'].idxmax(), 'species']}")
    display(croppedimage)
else:
    print('Nie wykryto zwierzęcia')

#### Pętla klasyfikacji

In [4]:
detector = Detector()
classifier = Classifier()

Using deepfaune-yolov8s_960 with weights at models\deepfaune-yolov8s_960.pt, in resolution 960x960
Using vit_large_patch14_dinov2.lvd142m with weights at models\deepfaune-vit_large_patch14_dinov2.lvd142m.v3.pt, in resolution 182x182
CUDA available


In [5]:
IMAGES_PATH = 'test_images/'
images_names = os.listdir(IMAGES_PATH)

print_list = ['-' for i in images_names]
ith_char = 0
print(''.join(print_list), end='\r')

results = pd.DataFrame({'image': [], 'detected_animal': []})

for img_name in images_names:
    croppedimage, category, box, count, humanboxes = detector.bestBoxDetection(IMAGES_PATH + img_name)

    if croppedimage is not None:
        cropped_tensor = torch.ones((1,3,CROP_SIZE,CROP_SIZE))
        cropped_tensor[0,:,:,:] =  classifier.preprocessImage(croppedimage)
        scores = classifier.model.predict(cropped_tensor)
        output = pd.DataFrame({'species':txt_animalclasses['pl'], 'score':scores[0][:]})
        output = output[output['score'] > 0.2]

        results.loc[len(results)] = [img_name, output.loc[output['score'].idxmax(), 'species']]
    else:
        results.loc[len(results)] = [img_name, 'Brak']
    
    print_list[ith_char] = '*'
    print(''.join(print_list), end='\r')
    ith_char += 1

results.to_csv('results/results_' + DFYOLO_NAME + '_' + BACKBONE + '.csv')

In [6]:
results

,image,detected_animal
0,2020-01-07 00-47-51.JPG,jeleń szlachetny
1,2022-09-05 13-57-18.JPG,lis
2,2022-09-06 19-34-11.JPG,Brak
3,2022-09-10 22-41-29.JPG,borsuk
4,2022-09-13 01-29-08.JPG,Brak
...,...,...
116,IMG_0607.JPG,żubr
117,IMG_0726.JPG,wilk
118,IMG_0800.JPG,kot
119,IMG_0814.JPG,lis


#### Sprawdzanie crop_size

In [ ]:
df182 = pd.read_csv('results/results_deepfaune-yolov8s_960_vit_large_patch14_dinov2.lvd142m_cropsize_182.csv')
df280 = pd.read_csv('results/results_deepfaune-yolov8s_960_vit_large_patch14_dinov2.lvd142m_cropsize_280.csv')
df560 = pd.read_csv('results/results_deepfaune-yolov8s_960_vit_large_patch14_dinov2.lvd142m_cropsize_560.csv')
df182.rename(columns={'detected_animal': 'detected_animal_182'}, inplace=True)
df280.rename(columns={'detected_animal': 'detected_animal_280'}, inplace=True)
df560.rename(columns={'detected_animal': 'detected_animal_560'}, inplace=True)

joined = pd.concat([df182, df280, df560], axis=1)[['image', 'detected_animal_182', 'detected_animal_280', 'detected_animal_560']]
joined['match'] = ((joined['detected_animal_182'] == joined['detected_animal_280']) \
                   & (joined['detected_animal_280'] == joined['detected_animal_560']) \
                   & (joined['detected_animal_182'] == joined['detected_animal_560']))

joined.loc[joined['match'] == False]